In [1]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.metrics import f1_score
import json
import time
import os
cv2.setNumThreads(0)

In [2]:
import json
import pandas as pd

# Load train metadata
with open('train_metadata.json', 'r') as f:
    train_data = json.load(f)

images_df = pd.DataFrame(train_data['images'])
annotations_df = pd.DataFrame(train_data['annotations'])
train_df = images_df.merge(annotations_df, on='image_id')

print(f'Train samples: {len(train_df)}')
print(f'Train columns: {train_df.columns.tolist()}')
print(f'Unique classes: {train_df["category_id"].nunique()}')

# Remap category_id to contiguous 0 to 15500 due to gaps
unique_cats = sorted(train_df['category_id'].unique())
cat_to_idx = {cat: idx for idx, cat in enumerate(unique_cats)}
idx_to_cat = {idx: cat for cat, idx in cat_to_idx.items()}
train_df['mapped_label'] = train_df['category_id'].map(cat_to_idx)
print(f'Mapping created: {len(cat_to_idx)} classes, max mapped: {train_df["mapped_label"].max()}')

# Load test metadata
with open('test_metadata.json', 'r') as f:
    test_data = json.load(f)

test_df = pd.DataFrame(test_data)
print(f'Test samples: {len(test_df)}')
print(f'Test columns: {test_df.columns.tolist()}')
print('Train head:', train_df.head())

Train samples: 665720
Train columns: ['file_name', 'image_id', 'license', 'category_id', 'genus_id', 'institution_id']
Unique classes: 15501
Mapping created: 15501 classes, max mapped: 15500
Test samples: 174052
Test columns: ['file_name', 'image_id', 'license']
Train head:                file_name    image_id  license  category_id  genus_id  \
0  000/00/00000__001.jpg  00000__001        0            0         1   
1  000/00/00000__002.jpg  00000__002        0            0         1   
2  000/00/00000__003.jpg  00000__003        0            0         1   
3  000/00/00000__004.jpg  00000__004        0            0         1   
4  000/00/00000__005.jpg  00000__005        0            0         1   

   institution_id  mapped_label  
0              52             0  
1              55             0  
2              52             0  
3              52             0  
4              52             0  


In [3]:
print('Max category_id in train_df:', train_df['category_id'].max())
print('Min category_id in train_df:', train_df['category_id'].min())
print('Number of unique category_id:', train_df['category_id'].nunique())
gaps = set(range(train_df['category_id'].min(), train_df['category_id'].max() + 1)) - set(train_df['category_id'].unique())
print('Number of gaps in category_id:', len(gaps))

Max category_id in train_df: 15504
Min category_id in train_df: 0
Number of unique category_id: 15501
Number of gaps in category_id: 4


In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
train_idx, val_idx = next(splitter.split(train_df, train_df['mapped_label']))
train_split = train_df.iloc[train_idx].reset_index(drop=True)
val_split = train_df.iloc[val_idx].reset_index(drop=True)

print(f'Train split size: {len(train_split)}')
print(f'Val split size: {len(val_split)}')
print(f'Val unique classes: {val_split["mapped_label"].nunique()}')

Train split size: 599148
Val split size: 66572
Val unique classes: 15417


In [5]:
# WeightedRandomSampler commented out per expert advice - use shuffle=True + class-weighted loss
# class_counts = Counter(train_split['category_id'])
# sample_weights = [1.0 / class_counts[cat] for cat in train_split['category_id']]
# sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
# print(f'Sampler stats - mean weight: {np.mean(sample_weights):.4f}, min weight: {np.min(sample_weights):.4f}, max weight: {np.max(sample_weights):.4f}')
print('Sampler disabled - using shuffle=True with weighted loss.')

Sampler disabled - using shuffle=True with weighted loss.


In [6]:
ROOT_TRAIN = 'train_images'
class HerbariumDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        import torch  # Import here for multiprocessing workers
        row = self.df.iloc[idx]
        img_path = os.path.join(ROOT_TRAIN, row['file_name'])
        img = cv2.imread(img_path)
        if img is None:
            raise ValueError(f'Image not found: {img_path}')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = row['mapped_label']
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
        return img, torch.tensor(label, dtype=torch.long)

# Quick check for first image
import os
first_path = os.path.join(ROOT_TRAIN, train_df.iloc[0]['file_name'])
print(f'First image exists: {os.path.exists(first_path)}')

print('HerbariumDataset defined with mapped labels and error on missing images.')

First image exists: True
HerbariumDataset defined with mapped labels and error on missing images.


In [7]:
train_transform = A.Compose([
    A.LongestMaxSize(max_size=224),
    A.PadIfNeeded(min_height=224, min_width=224, border_mode=cv2.BORDER_CONSTANT, value=[0,0,0]),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

val_transform = A.Compose([
    A.LongestMaxSize(max_size=224),
    A.PadIfNeeded(min_height=224, min_width=224, border_mode=cv2.BORDER_CONSTANT, value=[0,0,0]),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

print('Transforms: 224 + Flip + Normalize')

Transforms: 224 + Flip + Normalize


In [8]:
train_ds = HerbariumDataset(train_split, train_transform)
val_ds = HerbariumDataset(val_split, val_transform)

# Updated for speed: bs=32/64, workers=4, pin=True, prefetch=2
train_loader = DataLoader(
    train_ds,
    batch_size=32,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    persistent_workers=False,
    prefetch_factor=2
)
val_loader = DataLoader(
    val_ds,
    batch_size=64,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    persistent_workers=False,
    prefetch_factor=2
)
print(f'Train batches: {len(train_loader)}, Val batches: {len(val_loader)}')

Train batches: 18724, Val batches: 1041


In [9]:
import torch
from torchvision.models import efficientnet_v2_s
import torch.nn as nn
from collections import Counter
import numpy as np
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print(f'Train split size: {len(train_split)}')
n_classes = train_df['mapped_label'].nunique()
model = efficientnet_v2_s(weights='DEFAULT')
print('Classifier type:', type(model.classifier))
if isinstance(model.classifier, nn.Sequential):
    in_features = model.classifier[-1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(in_features, n_classes)
    )
    print('Replaced Sequential classifier with new head')
else:
    in_features = model.classifier.in_features
    model.classifier = nn.Linear(in_features, n_classes)
    print('Replaced direct Linear classifier with new head')
model = model.to(device)
# torch.compile disabled due to compilation error in environment
# if hasattr(torch, "compile"):
#     model = torch.compile(model, mode="reduce-overhead")
print(f'GPU allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB')
print(f'GPU reserved:  {torch.cuda.memory_reserved()/1024**3:.2f} GB')
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('high')
model = model.to(memory_format=torch.channels_last)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Model created: EfficientNet-V2-S with {total_params:,} trainable params')
# Balanced Softmax: compute log_freq for logit adjustment
counts = np.bincount(train_split['mapped_label'].values, minlength=n_classes).astype(np.float64)
log_freq = np.log(counts + 1e-6)
log_freq = torch.tensor(log_freq, dtype=torch.float32, device=device)
print(f'Log freq mean: {log_freq.mean().item():.4f}, max: {log_freq.max().item():.4f}')
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
# Fixed EMA (full decay)
class ModelEMA:
    def __init__(self, model, decay=0.999):
        self.model = model
        self.decay = decay
        self.shadow = {n: p.data.clone() for n,p in model.named_parameters() if p.requires_grad}
        self.backup = {}
    @torch.no_grad()
    def update(self):
        d = self.decay
        for n, p in self.model.named_parameters():
            if not p.requires_grad: continue
            self.shadow[n].mul_(d).add_(p.data, alpha=1.0 - d)
    @torch.no_grad()
    def apply_shadow(self):
        self.backup = {}
        for n, p in self.model.named_parameters():
            if not p.requires_grad: continue
            self.backup[n] = p.data.clone()
            p.data.copy_(self.shadow[n])
    @torch.no_grad()
    def restore(self):
        for n, p in self.model.named_parameters():
            if not p.requires_grad: continue
            p.data.copy_(self.backup[n])
ema = ModelEMA(model, decay=0.999)
# Optimizer and Scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=6)
scaler = torch.amp.GradScaler('cuda')
print('Setup complete: EfficientNet-V2-S, Balanced Softmax CE Loss, EMA (decay=0.999), AdamW, CosineAnnealingLR T_max=6, AMP ready.')
print('Weights dtype:', log_freq.dtype)

Using device: cuda
Train split size: 599148


Classifier type: <class 'torch.nn.modules.container.Sequential'>
Replaced Sequential classifier with new head
GPU allocated: 0.34 GB
GPU reserved:  0.35 GB
Model created: EfficientNet-V2-S with 40,034,269 trainable params
Log freq mean: 3.4938, max: 4.0604
Setup complete: EfficientNet-V2-S, Balanced Softmax CE Loss, EMA (decay=0.999), AdamW, CosineAnnealingLR T_max=6, AMP ready.
Weights dtype: torch.float32


In [10]:
num_epochs = 6  # Full training
best_f1 = 0
accum_steps = 2  # eff_bs = 32*2 = 64
optimizer.zero_grad(set_to_none=True)

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    train_loss = 0.0
    num_batches = 0
    epoch_start = time.time()

    for batch_idx, (imgs, labels) in enumerate(train_loader):
        imgs = imgs.to(device, non_blocking=True).to(memory_format=torch.channels_last)
        labels = labels.to(device, non_blocking=True)

        mixup_active = epoch > 1
        if mixup_active:
            lam = np.random.beta(0.2, 0.2)
            idx = torch.randperm(imgs.size(0), device=imgs.device)
            imgs2, labels2 = imgs[idx], labels[idx]
            imgs = lam * imgs + (1 - lam) * imgs2
            # For Mixup with Balanced: mixed hard-label CE on adjusted logits
            with torch.amp.autocast('cuda'):
                outputs = model(imgs)
                adjusted = outputs - log_freq.unsqueeze(0)
                loss1 = criterion(adjusted, labels)
                loss2 = criterion(adjusted, labels2)
                loss = lam * loss1 + (1 - lam) * loss2
        else:
            with torch.amp.autocast('cuda'):
                outputs = model(imgs)
                adjusted = outputs - log_freq.unsqueeze(0)
                loss = criterion(adjusted, labels)

        loss = loss / accum_steps
        scaler.scale(loss).backward()

        if (batch_idx + 1) % accum_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            ema.update()

        train_loss += loss.item() * accum_steps
        num_batches += 1

        if batch_idx % 500 == 0:
            elapsed = time.time() - epoch_start
            imgs_per_sec = ((batch_idx + 1) * train_loader.batch_size) / max(1e-6, elapsed)
            eta_min = ((len(train_loader) - batch_idx) * train_loader.batch_size / max(imgs_per_sec,1e-6)) / 60
            print(f'Epoch {epoch+1}, Batch {batch_idx}/{len(train_loader)}, Loss: {(loss.item()*accum_steps):.4f}, Img/s: {imgs_per_sec:.1f}, ETA: {eta_min:.1f}min', flush=True)
            print(f'GPU Mem Alloc: {torch.cuda.memory_allocated()/1024**3:.2f} GB, Reserved: {torch.cuda.memory_reserved()/1024**3:.2f} GB', flush=True)

    # Validation with EMA
    ema.apply_shadow()
    model.eval()
    all_probs, all_labels = [], []
    val_loss = 0.0; val_batches = 0; val_start = time.time()
    with torch.no_grad():
        for batch_idx, (imgs, labels) in enumerate(val_loader):
            imgs = imgs.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            labels = labels.to(device, non_blocking=True)
            with torch.amp.autocast('cuda'):
                outputs = model(imgs)
                adjusted = outputs - log_freq.unsqueeze(0)
                loss = criterion(adjusted, labels)
                val_loss += loss.item(); val_batches += 1
            probs = torch.softmax(adjusted, dim=1)
            all_probs.append(probs.cpu()); all_labels.append(labels.cpu())
            if batch_idx % 200 == 0:
                print(f'Val Batch {batch_idx}/{len(val_loader)}, Elapsed: {(time.time() - val_start)/60:.1f}min', flush=True)
    ema.restore()

    avg_train_loss = train_loss / max(num_batches,1)
    avg_val_loss = val_loss / max(val_batches,1)
    all_probs = torch.cat(all_probs); all_labels = torch.cat(all_labels)
    preds = all_probs.argmax(1)
    val_f1 = f1_score(all_labels.numpy(), preds.numpy(), average='macro')
    top1_acc = (preds == all_labels).float().mean().item()
    unique_preds = len(torch.unique(preds))
    top5_preds = Counter(preds.numpy()).most_common(5)
    epoch_time = time.time() - start_time
    print(f'Epoch {epoch+1}: Train Loss {avg_train_loss:.4f}, Val Loss {avg_val_loss:.4f}, Val F1 {val_f1:.4f}, Top1 Acc {top1_acc:.4f}, Unique Pred Classes {unique_preds}, Time {epoch_time/60:.2f}min', flush=True)
    print(f'Top 5 pred classes: {top5_preds}', flush=True)

    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), 'best_model_full.pth')
        torch.save({k: v.cpu() for k, v in ema.shadow.items()}, 'best_ema_full.pth')
        print(f'New best model saved! Best F1: {best_f1:.4f}')

    scheduler.step()

print(f'Training completed. Best Val F1: {best_f1:.4f}')

Epoch 1, Batch 0/18724, Loss: 10.1303, Img/s: 61.5, ETA: 162.4min


GPU Mem Alloc: 0.66 GB, Reserved: 3.15 GB


Epoch 1, Batch 500/18724, Loss: 9.9542, Img/s: 449.9, ETA: 21.6min


GPU Mem Alloc: 0.96 GB, Reserved: 3.75 GB


Epoch 1, Batch 1000/18724, Loss: 9.5318, Img/s: 458.5, ETA: 20.6min


...

Val Batch 800/1041, Elapsed: 0.7min


Val Batch 1000/1041, Elapsed: 0.9min


Epoch 6: Train Loss 2.4373, Val Loss 2.0241, Val F1 0.6428, Top1 Acc 0.6957, Unique Pred Classes 14169, Time 23.54min


Top 5 pred classes: [(7531, 16), (12975, 16), (14228, 15), (2625, 15), (15321, 15)]


New best model saved! Best F1: 0.6428
Training completed. Best Val F1: 0.6428


In [ ]:
# Test one image load first
sample_path = os.path.join('test_images', test_df.iloc[0]['file_name'])
img = cv2.imread(sample_path)
print(f'Sample test image loaded: shape {img.shape if img is not None else "None"}')

# Test Inference with TTA
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

class TestDataset(Dataset):
    def __init__(self, df, transform=None, root='test_images'):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.root = root
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.root, row['file_name'])
        img = cv2.imread(img_path)
        if img is None:
            raise ValueError(f'Test image not found: {img_path}')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
        return img

# Common base transforms (without Compose)
base_transforms = [
    A.LongestMaxSize(max_size=224),
    A.PadIfNeeded(min_height=224, min_width=224, border_mode=cv2.BORDER_CONSTANT, value=[0,0,0]),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
]

# TTA: 4 views for speed (original, hflip, rot90, rot270) - reduced from 8 to avoid timeout
tta_augs = [
    [],  # original
    [A.HorizontalFlip(p=1)],  # hflip
    [A.Rotate(90, p=1)],  # rot90
    [A.Rotate(270, p=1)]   # rot270
]
tta_transforms = [A.Compose(augs + base_transforms) for augs in tta_augs]

# Load best EMA model correctly
from torchvision.models import efficientnet_v2_s
import torch.nn as nn
new_model = efficientnet_v2_s(weights=None)
if isinstance(new_model.classifier, nn.Sequential):
    in_features = new_model.classifier[-1].in_features
    new_model.classifier = nn.Sequential(nn.Dropout(p=0.2, inplace=True), nn.Linear(in_features, n_classes))
else:
    in_features = new_model.classifier.in_features
    new_model.classifier = nn.Linear(in_features, n_classes)
# Load full base state_dict (includes buffers)
base_state = torch.load('best_model_full.pth', weights_only=False)
new_model.load_state_dict(base_state)
# Load EMA shadows and update parameters only
ema_state = torch.load('best_ema_full.pth', weights_only=False)
for key, value in ema_state.items():
    if key in new_model.state_dict():
        new_model.state_dict()[key].copy_(value)
new_model.to(device).to(memory_format=torch.channels_last)
new_model.eval()
print('Model loaded for inference.')

# Inference with TTA (reduced to 4x for speed)
all_test_probs = torch.zeros(len(test_df), n_classes, device='cpu')  # cpu to save GPU mem
num_tta = len(tta_transforms)
print(f'Starting TTA inference with {num_tta} views on {len(test_df)} images.')
with torch.no_grad():
    for tta_idx, tta_trans in enumerate(tta_transforms):
        print(f'TTA {tta_idx+1}/{num_tta} starting...')
        tta_start = time.time()
        tta_ds = TestDataset(test_df, transform=tta_trans)
        tta_loader = DataLoader(tta_ds, batch_size=128, shuffle=False, num_workers=2, pin_memory=True, prefetch_factor=2)
        tta_probs = torch.zeros(len(test_df), n_classes, device='cpu')
        num_batches = len(tta_loader)
        for i, imgs in enumerate(tta_loader):
            if i == 0:
                print(f'First batch loaded successfully, shape: {imgs.shape}')
            imgs = imgs.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            with torch.amp.autocast('cuda'):
                outputs = new_model(imgs)
                adjusted = outputs - log_freq.unsqueeze(0)
                probs = F.softmax(adjusted, dim=1)
            tta_probs[i*128:(i+1)*128] = probs.cpu()
            if i % 100 == 0:
                elapsed = time.time() - tta_start
                eta = (num_batches - i) * (elapsed / max(i, 1)) / 60
                print(f'TTA {tta_idx+1}, Batch {i}/{num_batches}, ETA: {eta:.1f}min')
        all_test_probs += tta_probs / num_tta
        tta_time = time.time() - tta_start
        print(f'TTA {tta_idx+1} completed in {tta_time/60:.1f}min')

print('All TTA completed. Generating submission...')
preds = all_test_probs.argmax(1)
category_ids = [idx_to_cat[p.item()] for p in preds]
submission = pd.DataFrame({'Id': range(len(test_df)), 'Predicted': category_ids})
submission.to_csv('submission.csv', index=False)
print('Submission saved with 4x TTA (fixed format: Id, Predicted).')
print(f'Sample: {submission.head()}')
print(f'Pred class distribution top 5: {Counter(category_ids).most_common(5)}')

Sample test image loaded: shape (1000, 666, 3)


Model loaded for inference.
Starting TTA inference with 4 views on 174052 images.
TTA 1/4 starting...


First batch loaded successfully, shape: torch.Size([128, 3, 224, 224])
TTA 1, Batch 0/1360, ETA: 36.4min


In [14]:
# Diagnose submission format
sample_sub = pd.read_csv('sample_submission.csv')
print('Sample submission head:')
print(sample_sub.head())
print('Sample columns:', sample_sub.columns.tolist())
print('Sample image_id dtype:', sample_sub['image_id'].dtype)
print('Sample image_id sample:', sample_sub['image_id'].iloc[0], type(sample_sub['image_id'].iloc[0]))

current_sub = pd.read_csv('submission.csv')
print('\nCurrent submission head:')
print(current_sub.head())
print('Current columns:', current_sub.columns.tolist())
print('Current image_id dtype:', current_sub['image_id'].dtype)
print('Current image_id sample:', current_sub['image_id'].iloc[0], type(current_sub['image_id'].iloc[0]))

print('\nTest_df head:')
print(test_df.head())
print('Test image_id dtype:', test_df['image_id'].dtype)

Sample submission head:
   Id  Predicted
0   0         42
1   1         42
2   2         42
3   3         42
4   4         42
Sample columns: ['Id', 'Predicted']


KeyError: 'image_id'